<a href="https://colab.research.google.com/github/oboog/dus-06_easyparking_ss22_ds_group7/blob/oliversbranch/Streamlit_obo_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install streamlit -q

In [8]:
#hier wird über writefile eine app.py Datei erstellt, dort können wir sowohl streamlit importieren als auch (Text)Inhalte ausgeben. 
#versuchen, nach writefile die Datei von Hannah einzubinden: 

%%writefile app.py
import streamlit as st
st.write("Hello World from Oliver")

Overwriting app.py


In [3]:
#entweder hier oder später pyngrok. das ist ein wrapper, so wie ich verstanden habe, ermöglicht es uns über tunneling eine Frontendansicht der app.py datei zu erstellen
# siehe Anleitung hier: https://www.youtube.com/watch?v=MUD-pBOnvdo&ab_channel=1littlecoder

! pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
#%%writefile app.py
import streamlit as st
import random
import pandas as pd
import pydeck as pdk

df_data = pd.read_csv('../techlabs_merging/merged_data.csv', index_col=0)
df_data = df_data.loc[:, ['x', 'y', 'type']]
st.title("Easy Parking")

address_query = st.text_input("Enter your destination", value='Cologne')

selected_types = st.multiselect(
     'What do you want to see around your destination?',
     df_data.type.unique(), default=df_data.type.unique())
    
type_mask = df_data.type.isin(selected_types)

# Filter data for selected types
df_data = df_data[type_mask]

@st.cache
def generate_random_hexcolor(type):
    return "#%06x" % random.randint(0, 0xFFFFFF)

def hex_color_to_rgb_list(hex_color):
    """
    Given a hex color like #00FF33, returns an array with decimal numbers like [0, 256, 0]
    """
    return list(int(hex_color.lstrip('#')[i:i+2], 16) for i in (0, 2, 4))

colors = {}

if df_data.type.nunique() > 0: # If user deselects all options, ignore showing color pickers for data points to avoid crash
    cols = st.columns(df_data.type.nunique())
    for i, type in enumerate(df_data.type.unique()):
        with cols[i]:
            hex_color = st.color_picker(type, value=generate_random_hexcolor(type))
            rgb_list = hex_color_to_rgb_list(hex_color)
            rgba_list = rgb_list + [200] # add fixed alpha value as last element
            colors[type] = rgba_list

layers = [
         pdk.Layer(
             'ScatterplotLayer',
             data=df_data[df_data['type']==type],
             get_position='[x, y]',
             get_color=f'{color}',
             get_radius=15,
         )
         for type, color in colors.items()
     ]

@st.cache # Don't query the geocode service for the same address again and again
def get_address_coordinate(query):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent='map')

    location = geolocator.geocode(query)
    return (location.longitude, location.latitude)

queried_location = get_address_coordinate(address_query)
df_queried_location = pd.DataFrame(queried_location , index=['x','y']).T

user_input_address = pdk.Layer(
    'ScatterplotLayer',
    data=df_queried_location,
    get_position='[x, y]',
    get_color='[100, 100, 100, 200]',
    get_radius=100,
)
layers.append(user_input_address)

st.pydeck_chart(pdk.Deck(
     map_style='mapbox://styles/mapbox/light-v9',
     initial_view_state=pdk.ViewState(
         longitude=queried_location[0],
         latitude=queried_location[1],
         zoom=14,
         pitch=0,
     ),
     layers=layers,
 ))

Overwriting app.py


In [5]:
from pyngrok import ngrok

In [6]:
ngrok.set_auth_token("2DlrIVxVNodhO45eVjVn7gvhIym_4TXTNM5PPduCG2QBegVVi")

In [7]:
!nohub streamlit run app.py --server.port 80 &
url = ngrok.connect(port = "80")
print(url)

/bin/bash: nohub: command not found
NgrokTunnel: "http://e820-34-139-187-128.ngrok.io" -> "http://localhost:80"


In [9]:
# hier noch eine weitere Anleitung "Another way of running a publicly accessible server using ngrok:"
#https://stackoverflow.com/questions/60571301/run-localhost-server-in-google-colab-notebook

# >>>> klappt bei mir auch nicht <<<<<


# !pip install pyngrok --quiet
# from pyngrok import ngrok

# #Terminate open tunnels if exist
#ngrok.kill()

# # Setting the authtoken (optional)
# # Get your authtoken from https://dashboard.ngrok.com/auth
# NGROK_AUTH_TOKEN = "2DlrIVxVNodhO45eVjVn7gvhIym_4TXTNM5PPduCG2QBegVVi"
# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Open an HTTPs tunnel on port 5000 for http://localhost:5000
# public_url = ngrok.connect(port="5000", proto="http", options={"bind_tls": True})
# print("Tracking URL:", public_url)

Tracking URL: NgrokTunnel: "http://40bd-34-139-187-128.ngrok.io" -> "http://localhost:80"


# Als nächstes folgenden Code für Google Maps Einbindung in den Code von Hannah integrieren


In [ ]:
#import webbrowser
#import geocoder
#loc = geocoder.ip('me').latlng
#street = "Kampstraße"
#current_geolocation = "{latitude}, {longitude}".format(latitude=loc[0], longitude=loc[1])
#destination = "{latitude}, {longitude}".format(latitude=loc[0]+1, longitude=loc[1]+1)
# #"{street}+{number},+{zip_code}+{city}".format(street=street, number=number,
                                                           #zip_code=zip_code, city=city)
# url = 'https://www.google.com/maps/dir/{start}/{end}'.format(start=current_geolocation, end=destination)
